# Файл отладки

In [1]:
from pathlib import Path
from tqdm import tqdm
import json
import torch
import torch.nn as nn
from torchsummary import summary
import torchvision.transforms.v2 as transforms_v2
from torchvision.transforms.v2 import InterpolationMode

# Import Utils.
from torch.utils.data import DataLoader
from src.utils.metrics import CombinedLoss, dice_coefficient, iou_score, pixel_accuracy

# Import Datasets.
from src.datasets.TinyImageNetDataset import TinyImageNetDataset
from src.datasets.MoonSegmentBinaryDataset import MoonSegmentationDataset

# Import models.
from src.models.model_utilizer import load_net
from src.models.customResNet import customResNet
from src.models.customUNet import customUNet
from src.models.customResNetUNet import customResNetUNet

from src.main import set_seed
from src.train import MetricsHistory

In [2]:
if torch.cuda.is_available():
    print("CUDA is available!")
    print(f"Number of GPUs: {torch.cuda.device_count()}")
    print(f"Current GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"CUDA Version used by PyTorch: {torch.version.cuda}")
else:
    print("CUDA is not available. PyTorch is using CPU.")

CUDA is not available. PyTorch is using CPU.


Пути к файлам конфигурации.

In [3]:
# Read config-files.
hyperparameters_dir = Path("./src/hyperparameters/")

config_path = hyperparameters_dir / "config.json"
assert config_path.exists(), f"Config not found: {config_path}"
with open(config_path, "r") as f:
    general_config = json.load(f)

set_seed(general_config["seed"])
device = torch.device(general_config["device"].lower() if torch.cuda.is_available() else 'cpu')

checkpoints_dir = Path(general_config["checkpoints_dir"])
logs_dir = Path(general_config["logs_dir"])

## Отладка датасетов

In [4]:
from enum import IntEnum

# user-defined index.
class usid(IntEnum):
    customResNet = 0
    customUNet = 1
    customResNetUNet = 2
    customResNetUNet_pretrained = 3

# Имена моделей.
model_names = [e.name for e in usid]

In [5]:
metadata_list = []
train_log_list = []
train_log_df_list = []
checkpoints_file_list = []

for name in model_names:
    train_log_path = logs_dir / f"{name}.json"
    assert train_log_path.exists(), f"Log not found: {train_log_path}"
    with open(train_log_path, "r") as f:
        train_log_full = json.load(f)
    metadata = train_log_full['metadata']
    
    checkpoints_file = checkpoints_dir / f"best_{name}.pth"
    assert checkpoints_file.exists(), f"Checkpoints not found: {checkpoints_file}"
    
    metadata_list.append(metadata)
    train_log_list.append(train_log_full['train_log'])
    checkpoints_file_list.append(checkpoints_file)

In [3]:
model_name = "customResNet"

# Read config-files.
hyperparameters_dir = Path("./src/hyperparameters/")

config_path = hyperparameters_dir / "config.json"
assert config_path.exists(), f"Config not found: {config_path}"
with open(config_path, "r") as f:
    general_config = json.load(f)

set_seed(general_config.get("seed"))

model_config_path = hyperparameters_dir / f"{model_name}-config.json"
assert model_config_path.exists(), f"Config not found: {model_config_path}"
with open(model_config_path, "r") as f:
    model_config = json.load(f)

dataset_config_path = hyperparameters_dir / f"{model_config.get('dataset_name')}-config.json"
assert dataset_config_path.exists(), f"Config not found: {dataset_config_path}"
with open(dataset_config_path, "r") as f:
    dataset_config = json.load(f)
    
device = torch.device(general_config.get("device").lower() if torch.cuda.is_available() else 'cpu')
data_path = Path(general_config.get("data_dir")) / model_config.get("dataset_name")

In [4]:
checkpoints_file = Path('./checkpoints/customResNet/pretrained_customResNet_4x2_classes_10.pth')

selected_classes = dataset_config.get("selected_classes")
n_classes = len(selected_classes)

mdl_input_size = model_config.get('input_size')

if checkpoints_file is None:
        pretrained_flag = False
else:
        pretrained_flag = True

model, _, _ = customResNet(
        layers_config = model_config.get("layers_num")*[model_config.get("block_size")],
        in_channels = mdl_input_size[0],
        layer0_channels = model_config.get("output_channels") // 2**(model_config.get("layers_num") - 1),
        num_classes = n_classes,
        pretrained = pretrained_flag,
        checkpoints_file = checkpoints_file,
        device = device
        )
model = model.to(device)
model.eval()

test_input = torch.randn(1, *mdl_input_size)
print(model_config.get("layers_num")*[model_config.get("block_size")])
model_size = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Model size: {model_size}")

summary(model, tuple(mdl_input_size))


Restoring checkpoint:  checkpoints\customResNet\pretrained_customResNet_4x2_classes_10.pth
[2, 2, 2, 2]
Model size: 889732
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 18, 64, 64]           2,646
       BatchNorm2d-2           [-1, 18, 64, 64]              36
              ReLU-3           [-1, 18, 64, 64]               0
         MaxPool2d-4           [-1, 18, 64, 64]               0
            Conv2d-5           [-1, 18, 32, 32]             324
       BatchNorm2d-6           [-1, 18, 32, 32]              36
            Conv2d-7           [-1, 18, 32, 32]           2,916
       BatchNorm2d-8           [-1, 18, 32, 32]              36
              ReLU-9           [-1, 18, 32, 32]               0
           Conv2d-10           [-1, 18, 32, 32]           2,916
      BatchNorm2d-11           [-1, 18, 32, 32]              36
             ReLU-12           [-1, 18, 32, 

In [5]:
mean_norm = [0.485, 0.456, 0.406]
std_norm = [0.229, 0.224, 0.225]

val_transforms = transforms.Compose([
    transforms.Resize(tuple(mdl_input_size[1:])),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean_norm, std=std_norm)
])
val_loader = DataLoader(
    TinyImageNetDataset(
        data_path = data_path,
        split = "val",
        transform = val_transforms,
        selected_classes = selected_classes
        ),
    batch_size=model_config.get("batch_size"),
    shuffle=False,
    num_workers=model_config.get("workers"),
    pin_memory=True)
print(f"Valid. size: {len(val_loader.dataset)}")

Valid. size: 500


In [6]:
loss_func = nn.CrossEntropyLoss().to(device)

metrics = MetricsHistory()
metrics.initialize_metrics(
    ['loss', 'accuracy'],
    ['val']
    )

with torch.no_grad():
    for data_tuple in tqdm(val_loader, desc="Val"):

        inputs, gt = data_tuple[0].to(device), data_tuple[1].to(device)

        output = model(inputs)
        
        loss = loss_func(output, gt)
        
        predicted = torch.argmax(output.detach(), dim=1)
        correct = gt.detach()
        
        metrics.update_metrics(
                    split = "val",
                    batch_size = inputs.size(0),
                    loss = loss.item(),
                    accuracy = float((predicted == correct).sum()) / len(correct))
        
        metrics.log_epoch_history(['val'])
metrics.print_metrics(['val'])

Val:   0%|          | 0/1 [00:00<?, ?it/s]d:\git\ITMO\DL\custom-U-Net\.venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
Val: 100%|██████████| 1/1 [00:06<00:00,  6.13s/it]

Valid. loss: 1.0364, accuracy: 0.6440


# U-Net

In [7]:
model = customUNet(
    in_channels=3,
    out_channels=1,
    features=[18, 36, 72, 144]
    )
model = model.to(device)
model.eval()

test_input = torch.randn(1, 3, 128, 128).to(device)
test_output = model(test_input)

print(f"   Вход:  {test_input.shape}")
print(f"   Выход: {test_output.shape}")

Encoder features by level: [18, 36, 72, 144]
   Вход:  torch.Size([1, 3, 128, 128])
   Выход: torch.Size([1, 1, 128, 128])


In [8]:
model = customResNetUNet(
    in_channels=3,
    out_channels=1,
    features=[18, 36, 72, 144],
    backbone_layers_config = 4*[2],
    backbone_layer0_channels = 18,
    backbone_pretrained = False,
    backbone_checkpoints_file = None,
    device = device
    )
model = model.to(device)
model.eval()

test_input = torch.randn(1, 3, 128, 128).to(device)
test_output = model(test_input)

print(f"   Вход:  {test_input.shape}")
print(f"   Выход: {test_output.shape}")

Encoder features by level: [18, 36, 72, 144]
   Вход:  torch.Size([1, 3, 128, 128])
   Выход: torch.Size([1, 1, 128, 128])
